<a href="https://colab.research.google.com/github/andyarnell/sepal_mgci/blob/master/SDG_15_4_2_Sub_B_Default_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SDG 15.4.2 Sub-indicator B: Calculate Global Default Values**

* This script allows batch processing for this indicator for all countries.

* Output is a combined excel file on your Google Drive.

* Runs on the cloud using [Google Colab](https://research.google.com/colaboratory/faq.html)

* Requires: [Google Earth Engine](https://earthengine.google.com/) (GEE) account and project and access to Google Drive


### 1) Install required packages

In [3]:
# to automatically reload modules.
%load_ext autoreload

# Set to reload all modules before executing code.
%autoreload 2

# Function to install a package if it's not already installed
def install_if_not_exists(package_name):
    try:
        __import__(package_name)
        print(f"{package_name} is already installed.")
    except ImportError:
        !pip install -q {package_name}
        print(f"{package_name} has been installed.")

# List of packages to install if not already installed
packages_to_install = ['ipyvuetify','ee', 'unidecode', 'google-api-python-client',
                      'google-auth-httplib2', 'google-auth-oauthlib']

# Install necessary packages
for package in packages_to_install:
    install_if_not_exists(package)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ipyvuetify is already installed.
ee is already installed.
unidecode is already installed.
google-api-python-client has been installed.
google-auth-httplib2 has been installed.
google-auth-oauthlib has been installed.


### 2) Access GitHub repository
Clones repository for SDG 15.4.2 into colab.
Provides functions and lookup tables etc.

In [4]:
# Change the current working directory to "/content" for cloning the repo into.
%cd "/content"

# Clone the GitHub repository "sepal_mgci" into the current directory.
# NB 'fatal' error on reruns are typically just saying it already exists
# !git clone https://github.com/sepal-contrib/sepal_mgci

!git clone https://github.com/andyarnell/sepal_mgci

# Change working directory to the cloned sepal_mgci github repository
%cd "/content/sepal_mgci"

/content
fatal: destination path 'sepal_mgci' already exists and is not an empty directory.
/content/sepal_mgci


### 3) Setup Google Earth Engine
Launches access request pop up window

In [5]:
# Google Earth Engine project
gee_project_name = "ee-andyarnellgee" # "insert cloud project here"  # a registered cloud project (if unsure of name see pic here: https://developers.google.com/earth-engine/cloud/assets)

import ee # google earth engine

ee.Authenticate()

ee.Initialize(project=gee_project_name) # NB gee project name is defined in parameters section

### 4) Setup Google Drive
Launches access request pop up window

In [6]:
# for accessing google drive
from google.colab import auth, drive
from googleapiclient.discovery import build

drive.mount('/content/drive')

Mounted at /content/drive


### 5) Import remaining packages

In [7]:
from component.scripts.colab_imports import * # import all packages listed in the imports.py script

Imports complete


### 6) Set parameters


Input parameters

In [8]:

# Admin boundaries asset
admin_asset_id = "FAO/GAUL/2015/level0" # administrative units feature collection

admin_asset_property_name = "ADM0_NAME" # property/column name for selecting admin boundaries (e.g. ISO3 code or country name)


# Land cover assets

# For Sub-indicator B (sub_b), we need to set the following structure
sub_b_year = {
    "baseline": {
        "base": {
            "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2000",
            "year": 2000,
        },
        "report": {
            "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2015",
            "year": 2015,
        },
    },
    # And the reporting year
    2: {"asset": "users/amitghosh/sdg_module/esa/cci_landcover/2018", "year": 2018},
}


Output parameters
- NB if no CSVs are being created, check that 'export = True' below.

---



In [9]:
final_report_folder = "sdg_15_4_2_B_combined_report" # folder name in Google Drive for final output (if doesnt exist creates one)

final_report_name = "sdg_15_4_2_B_default_global.xlsx" # file name for final excel output

# export GEE tasks or not
export = True # default: True. Set to False if debugging or limiting accidental re-exporting of tasks

# prints more messages
debug = False # default: False. Set to True if debugging code

Temporary output parameters


In [10]:
stats_csv_folder = "sdg_15_4_2_B_csvs" # for storing stats tables exported from GEE for each admin boundary/AOI

excel_reports_folder = "sdg_15_4_2_B_reports" # for storing formatted excel tables for each admin boundary/AOI

drive_home ="/content/drive/MyDrive/" # Google Drive location. Don't change unless you know this is incorrect

error_log_file_path = drive_home + excel_reports_folder + "/"+"1_error_log" +".csv" # for storing errors


### 7) Setup inputs for processing


Create list of boundaries to process

In [11]:
# admin boundary feature collection
admin_boundaries = ee.FeatureCollection(admin_asset_id)

# list to process
list_of_countries = admin_boundaries.aggregate_array(admin_asset_property_name).getInfo()

print ("Length of admin boundaries to process", len(list_of_countries))

list_of_countries = list(set(list_of_countries)) # remove dupicates

print ("Length of distinct admin boundaries to process", (len(set(list_of_countries))))


Length of admin boundaries to process 292
Length of distinct admin boundaries to process 276


Read the default land cover remapping table and convert it to a dictionary

In [12]:
default_map_matrix = map_matrix_to_dict(LC_MAP_MATRIX) # LC_MAP_MATRIX stores the path to the remapping table

Set the default transition matrix file path.

In [13]:
default_transition_matrix_path = TRANSITION_MATRIX_FILE #  location of the CSV table defining which land cover transitions are “degradation” etc.
print(default_transition_matrix_path) # view the file path

/content/sepal_mgci/component/parameter/transition_matrix.csv


Select years of land cover to process

In [14]:
# extracts the years from the b_years dictionary
years = get_b_years(sub_b_year)
years

[({'asset': 'users/amitghosh/sdg_module/esa/cci_landcover/2000', 'year': 2000},
  {'asset': 'users/amitghosh/sdg_module/esa/cci_landcover/2015', 'year': 2015},
  {'asset': 'users/amitghosh/sdg_module/esa/cci_landcover/2018',
   'year': 2018})]

### 8) Calculate area statistics by country
* Runs for each country and each mountain biobelt
* Gets area of land cover reclassified into the 10 SEAM classes
* Repeat for each year specified


In [15]:
# you can monitor your GEE tasks here : https://code.earthengine.google.com/tasks

create_folder_if_not_exists(stats_csv_folder) # to store outputs in google drive

counter=0 # starting place of counter used to keep track of number of tasks that are being run

for aoi_name in list_of_countries:

  aoi = admin_boundaries.filter(ee.Filter.eq(admin_asset_property_name,aoi_name))#.first()

  process = ee.FeatureCollection([
      ee.Feature(
          None,
          reduce_regions(
              aoi,
              remap_matrix=default_map_matrix,
              rsa=False,
              dem=DEM_DEFAULT, #default digital elevation model (DEM). Relevant for the real surface area (RSA) implementation.
              lc_years= year,
              transition_matrix=default_transition_matrix_path # a matrix of transitions between land cover classes. Used to assess if change between two inputs is to a different (e.g. degraded) state
          )
      ).set("process_id", "_".join([str(y["year"]) for y in year]))
  for year in years # creates GEE images for each year listed and counts areas under different transitions Subindicator B. Images to run are in the 'b_years" dictionary (above)
  ])
  #make name acceptable for running tasks (i.e., removes special characters)
  task_name = str(sanitize_description(unidecode(aoi_name)))

  task = ee.batch.Export.table.toDrive(
      **{  #asterisks unpack dictionary into keyword arguments format
          "collection": process,
          "description": task_name,
          "fileFormat": "CSV",
          "folder":stats_csv_folder
      }
  )

  counter+=1

  print (f"\r process {counter}/{len(list_of_countries)} {aoi_name} ", end="") #print in place (remove \r and end="" for verbose version)

  if export:
    task.start()



Folder 'sdg_15_4_2_B_csvs' already exists.
 process 276/276 Liechtenstein 

### 9) Read and translate results into report tables for each country

#####NOTE: you will need to wait until results files for each country have been created in your google drive (from previous step).
- see here to monitor the tasks https://code.earthengine.google.com/tasks
- once tasks are complete, you can run the cell below

This cell formats individual excel reports for each country.
See Error_log.csv for missing files/errors

In [18]:
# Initialize the counter
counter = 0

# to store outputs in google drive
create_folder_if_not_exists(excel_reports_folder)

# Loop over each AOI name in the list of countries
for aoi_name in list_of_countries:
    counter += 1

    # Clean the AOI name
    aoi_name_clean = str(sanitize_description(unidecode(aoi_name)))

    # Construct the file path for the stats CSV file
    stats_csv_file = aoi_name_clean + ".csv"
    stats_csv_file_path = os.path.join(drive_home, stats_csv_folder, stats_csv_file)

    message = f"Process {counter}, {stats_csv_file}"

    try:
        # Read the results from the CSV file and parse it to a dictionary
        dict_results = read_from_csv(stats_csv_file_path)

        # details = {
        #     "geo_area_name": aoi_name,
        #     "ref_area": " ",
        #     "source_detail": " ",
        # }

        sub_b_reports = []
        reporting_years_sub_b = get_reporting_years(sub_b_year, "sub_b")
        _, sub_b_years = get_sub_b_items(reporting_years_sub_b)

        for year in sub_b_years:
            print(f"Reporting {year} for sub_b")
            # Get year label for the report
            parsed_df = parse_to_year(dict_results, year)
            sub_b_reports.append(
                sub_b.get_reports(
                    parsed_df,
                    year,
                    geo_area_name = aoi_name,
                    ref_area = "",
                    source_detail = "",
                    transition_matrix = default_transition_matrix_path
                ))
        # sub b reports
        er_mtn_dgrp_df = pd.concat([report[0] for report in sub_b_reports])
        er_mtn_dgda_df = pd.concat([report[1] for report in sub_b_reports])




        # Define the output report file path
        report_file_path = os.path.join(drive_home, excel_reports_folder, aoi_name_clean + ".xlsx")

        # This will create the excel file with the reports
        with pd.ExcelWriter(report_file_path) as writer:
            er_mtn_dgda_df.to_excel(writer, sheet_name="Table4_ER_MTN_DGRDA", index=False)
            er_mtn_dgrp_df.to_excel(writer, sheet_name="Table5_ER_MTN_DGRDP", index=False)

            for sheetname in writer.sheets:
                worksheet = writer.sheets[sheetname]
                for col in worksheet.columns:
                    max_length = 0
                    column = col[0]
                    for cell in col:
                        try:
                            if len(str(cell.value)) > max_length:
                                max_length = len(cell.value)
                        except:
                            pass
                    adjusted_width = max(max_length, len(str(column.value))) + 4
                    worksheet.column_dimensions[get_column_letter(column.column)].width = (
                        adjusted_width
                    )

                    # Align "obs_value" column to the right
                    if "OBS" in column.value:
                        for cell in col:
                            cell.alignment = Alignment(horizontal="right")


    except Exception as e:
        # If an error occurs, catch the exception and handle it
        message = f"process {counter}, {stats_csv_file}, Error: {e}"

        # Get the current time
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Write the error message and file name to the error log file
        error_info = pd.DataFrame([[stats_csv_file, str(e), current_time]], columns=['File Name', 'Error Message', 'Time'])

        mode = 'w' if not os.path.exists(error_log_file_path) else 'a'
        header = False if os.path.exists(error_log_file_path) else True

        # Append or write to the error log file
        error_info.to_csv(error_log_file_path, mode=mode, header=header, index=False)

    print(message)


Folder 'sdg_15_4_2_B_reports' already exists.
process 1, Timor-Leste.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_B_csvs/Timor-Leste.csv'
process 2, Ethiopia.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_B_csvs/Ethiopia.csv'
Reporting {'baseline': (2000, 2015)} for sub_b
Reporting {'report': [2015, 2018]} for sub_b
Process 3, Andorra.csv
process 4, Togo.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_B_csvs/Togo.csv'
Reporting {'baseline': (2000, 2015)} for sub_b
Reporting {'report': [2015, 2018]} for sub_b
Process 5, British Virgin Islands.csv
process 6, Cameroon.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_B_csvs/Cameroon.csv'
process 7, Paracel Islands.csv, Error: [Errno 2] No such file or directory: '/content/drive/MyDrive/sdg_15_4_2_B_csvs/Paracel Islands.csv'
process 8, Niue.csv, Error: [Errno 2] No such file or directory: '/conte

### 10) Combine excel report files into one

Make a list of files to combine
- Note:  only combines excel files that are present in your google drive (from Step 9).

In [19]:
# Directory path where Excel reports are stored
directory_path = os.path.join(drive_home, excel_reports_folder)

# List files in the directory with '.xlsx' extension
files = [file for file in os.listdir(directory_path) if file.endswith('.xlsx')]

# Create a list of full file paths
full_file_paths = [os.path.join(directory_path, file) for file in files]

# Print the number of Excel files found in the folder
print(f"Number of Excel files in folder: {len(full_file_paths)}")

# folder to store outputs in google drive
create_folder_if_not_exists(final_report_folder)

# File path for the combined final report
reports_combined_file_path = os.path.join(drive_home, final_report_folder, final_report_name)


Number of Excel files in folder: 37
Folder 'sdg_15_4_2_B_combined_report' already exists.


##### Run function to combine into a single report

In [20]:
append_excel_files(file_paths=full_file_paths,num_sheets=3,output_file_path=reports_combined_file_path)

print (f"\n Complete! Output file for SDG 15.4.2 Component B here: {reports_combined_file_path}")

Processing 37/37: /content/drive/MyDrive/sdg_15_4_2_B_reports/Portugal.xlsx
 Complete! Output file for SDG 15.4.2 Component B here: /content/drive/MyDrive/sdg_15_4_2_B_combined_report/sdg_15_4_2_B_default_global.xlsx
